Offline Lambda
---
<img src="illustration.png" width="400">
---
The backup digram for `TD(λ)`. If `λ = 0`, then the overall update reduces to its first component, the one-step TD update, whereas if `λ = 1`, then the overall update reduces to its last component, the Monte Carlo update.
Update Rule
---
<img src="G_t.png" width="250">
<img src="offline_lambda.png" width="500">